# Demo: Peptide Prediction with BigQuery ML

### Supporting research: Covid19 and beyond for vaccine candidates 

BigQuery ML enables users to create and execute machine learning models in BigQuery using SQL queries. The goal is to democratize machine learning by enabling SQL practitioners to build models using their existing tools and to increase development speed by eliminating the need for data movement.
In this tutorial, you use the sample Covid19 dataset for BigQuery.
Comments & Feedback @jigmehta
## Objectives
In this tutorial, you use:
+ BigQuery `ML.CREATE` to create a classification model using the `CREATE MODEL` statement
+ The `ML.EVALUATE` function to evaluate the ML model
+ Use `ML.TRANSFORM`feature engineering functions to improve model performance
+ The `ML.PREDICT` function to make predictions using the ML model

Jupyter magics are notebook-specific shortcuts that allow you to run commands with minimal syntax. Jupyter notebooks come with many [built-in commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html). The BigQuery client library, `google-cloud-bigquery`, provides a cell magic, `%%bigquery`. The `%%bigquery` magic runs a SQL query and returns the results as a pandas `DataFrame`.

## Step 1: Explore Peptidic Epitope Data
For our coronavirus subset of data, let's examine data first:
+ Following query shows information about antigen.
+ Observe specific antigen that might of more interest to our study for coronavirus.
+ Can you find it?


In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
print("GCP project ID:" + GCP_PROJECT_ID)

In [ ]:
# Define project__id variable
project_id = 'covid-19-271622'

In [ ]:
%%bigquery --project $project_id
SELECT * 
FROM `corona.antigen`
LIMIT 10

Visually inspect number of epitopes by antigen name in our dataset.

In [ ]:
%%bigquery epitopes --project $project_id
SELECT
 Antigen_Name,
 sum(__Epitopes) AS epitope
FROM `corona.antigen`
GROUP BY Antigen_Name
ORDER BY epitope DESC

In [ ]:
epitopes.plot(kind='bar', x='Antigen_Name', y='epitope');

In [ ]:
# Following query shows epitope detail for those linear peptides whose parent protein
# came from Spike glycoprotein, which is our interests to investigate for possible antigen
# to protect against coronavirus


In [ ]:
%%bigquery --project $project_id
SELECT Epitope_ID, Object_Type, Description, Starting_Position,
            Ending_Position, Antigen_Name,Parent_Protein, Organism_Name
FROM `corona.epitope` 
WHERE Parent_Protein = 'Spike glycoprotein'
LIMIT 10

As you can see from the resulting query, we have a set of peptides generated from parent antigen protein. Now, we need to identify how these peptides bind with HLA molecule.

Following query shows HLA-peptide binding affinity information. The goal is to leverage machine learning models to predict binding affinity for many different and new peptides with HLA MHC Class-I molecules so that testing for vaccine candidates can be prioritized to accelerate solutions.


In [ ]:
%%bigquery --project $project_id
SELECT Reference_ID, Description, Allele_Name,MHC_allele_class, Assay_Group,
       Qualitative_Measure, Quantitative_measurement 
FROM `corona.mhc_ligand`
WHERE Epitope_ID = 12967 


In our dataset there are multiple of alleles and, for each allele we have epitope binding affinity measure. Lete inspect how epitopes and allele bindings are distributed in the dataset. This is one way to short list possible allele to consider for peptide binding research.

We will use dataframe to hold query result to build our visualization.

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
sql = """
SELECT Qualitative_Measure, 
       Allele_Name, 
       count(1) as count 
FROM `corona.mhc_ligand`
GROUP BY  1,2
ORDER BY 3 DESC
"""
df = client.query(sql).to_dataframe()
df.head()

In [ ]:
pivot_table = df.pivot(index='Allele_Name', columns='Qualitative_Measure', values='count')
pivot_table.plot(kind='bar', stacked=True, figsize=(15, 7));

Based on data insepction, it seems like for coronavirus data set, HLA-A-01-01, HLA-A-02-01, HLA-A-02-02, HLA-A-02-03 and HLA-A-020-06 alleles are most likely candidates for MHC-I molecules.

It is important to narrow down focus peptides and HLA binding to speed up testing on most probable vaccine candidate. Lets identify set of peptides that binds well with HLA, you can further narrow it to specific HLA as well. Following qury execution suggest that peptides with length of 9 and 10 are having better binding affinity. We should focus our research on those for now. Of course, more factors, features as well as 3D structure of aplha/beta components of HLA+Peptide bindings are useful for more complex modeling. 

In [ ]:
sql = """
SELECT Qualitative_Measure, 
       length(Description) as Peptide_Length, 
       count(1) as count 
FROM `corona.mhc_ligand`
GROUP BY  1,2
ORDER BY 3 DESC
"""
df = client.query(sql).to_dataframe()
df.head()

In [ ]:
pivot_table = df.pivot(index='Peptide_Length', columns='Qualitative_Measure', values='count')
pivot_table.plot(kind='bar', stacked=False, figsize=(15, 7));

Lets build feature table that translate Peotide Amino Acid sqquence into positioning columns. Explore amino acid properties table, it is prepared with allocation of acid sequnece number as well as additional properites that you can leverage for cross feature engineering.

In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `hla_peptide_generic.amino_acid`

We will examine pre-built feature table called mhc_qual_feature which has column presentation of 9 and 10 letter (MER) peptide sequence which are most suitable for MHC-1 binding. Table also contains qualitative measure and quantitative measure.

+ Peptides are transformed into column A1 to A10 based on amino acid position value in a chain
+ result_score is quantitative measure: lower the number, higher binding affinity
+ result columns is binary classification based on 5 class result_quality score

In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `corona.mhc_qual_feature`
LIMIT 10

## Step 2: Build First Classification Model
Building ML models with BigQuery is as simple as writing SQL statements; makes ML modeling accessible to even SQL developers and analysts. We will create a simple classification model to predict for a given peptide if there is strong binding affinity with certain HLA Allele.

Following statement creates a classification model using logistic regression. We are selecting feature columns of Allele and amino acid positioning within a peptide to classify if a peptide is a good candidate for vaccine testing.

+ Note: we are filtering data for peptides with length of 9 or 10 mers only. Also, since we run multiple samples, we are randomizing samples by 80% of data for learning.


In [ ]:
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `corona.Classification_model_bq1`
OPTIONS
(
model_type='logistic_reg',
input_label_cols=['result_quality']
)
AS
SELECT
 result_quality, Allele, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10
 FROM
  `corona.mhc_qual_feature`
 WHERE length(peptide) IN (9,10)
 AND rand() < 0.8

The query takes several minutes to complete. After the first iteration is
complete, your model (`Classification_model_bq1`) appears in the navigation panel of the
BigQuery web UI. Because the query uses a `CREATE MODEL` statement to create a
table, you do not see query results. The output is an empty `DataFrame`.

## Get training statistics

To see the results of the model training, you can use the
[`ML.TRAINING_INFO`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train)
function, or you can view the statistics in the BigQuery web UI.
In this tutorial, you use the `ML.TRAINING_INFO` function.

A machine learning algorithm builds a model by examining many examples and
attempting to find a model that minimizes loss. This process is called empirical
risk minimization.

Loss is the penalty for a bad prediction &mdash; a number indicating
how bad the model's prediction was on a single example. If the model's
prediction is perfect, the loss is zero; otherwise, the loss is greater. The
goal of training a model is to find a set of weights that have low
loss, on average, across all examples.

To see the model training statistics that were generated when you ran the
`CREATE MODEL` query:

In [ ]:
%%bigquery --project $project_id
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `corona.Classification_model_bq1`)

In [ ]:
%%bigquery loss_curve --project $project_id
SELECT
  iteration, loss, eval_loss
FROM
  ML.TRAINING_INFO(MODEL `corona.Classification_model_bq1`)
ORDER BY iteration ASC

In [ ]:
loss_curve.plot(x='iteration');

The `loss` column represents the loss metric calculated after the given iteration
on the training dataset. Since you performed a logistic regression, this column
is the [log loss](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression).
The `eval_loss` column is the same loss metric calculated on
the holdout dataset (data that is held back from training to validate the model).

For more details on the `ML.TRAINING_INFO` function, see the
[BigQuery ML syntax reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train).

## Evaluate your model

After creating your model, you evaluate the performance of the classifier using
the [`ML.EVALUATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-evaluate)
function. You can also use the [`ML.ROC_CURVE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-roc)
function for logistic regression specific metrics.

A classifier is one of a set of enumerated target values for a label. For
example, in this tutorial you are using a binary classification model that
detects transactions. The two classes are the values in the `label` column:
`0` (no transactions) and not `1` (transaction made).

To run the `ML.EVALUATE` query that evaluates the model:

In [ ]:
%%bigquery --project $project_id
SELECT
  *
FROM ML.EVALUATE(MODEL `corona.Classification_model_bq1`)

In [ ]:
%%bigquery --project $project_id
SELECT roc_auc,
       CASE WHEN roc_auc > .85 THEN 'good'
            WHEN roc_auc > .7 THEN 'fair'
            WHEN roc_auc > .5 THEN 'not great'
            ELSE 'poor' END AS model_quality
FROM ML.EVALUATE(MODEL `corona.Classification_model_bq1`)

## Step 3: Improve Model Performance with Feature Engineering
BigQuery offers many [transform / preprocessing](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions) functions for feature engineering on data. Advantage of transform functions is that once you build a model with preprocessing as part of model definition, prediction data does not need to be prepared as the model will apply transformation for the input. Lets see one example of a transform feature and rebuild our model to check if we get better model performance.

Following statement will create another classification model with preprocessing of the result score to normalize its deviation with respect to min-max value of an attribute.


In [ ]:
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `corona.Classification_model_bq2`
TRANSFORM (result_quality, Allele, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, 
           ML.MIN_MAX_SCALER(result_score) OVER() AS rs
)
OPTIONS
(
model_type='logistic_reg',
input_label_cols=['result_quality']
)
AS
SELECT result_quality, Allele, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, result_score
FROM `corona.mhc_qual_feature`
WHERE length(peptide) IN (9,10)
AND rand() < 0.8

The query takes several minutes to complete. After the first iteration is
complete, your model (`Classification_model_bq2`) appears in the navigation panel of the
BigQuery web UI. The output is an empty `DataFrame`.

Check model performance after feature engineering, run the `ML.EVALUATE` query that evaluates the model:

In [ ]:
%%bigquery loss_curve2 --project $project_id
SELECT
  iteration, loss, eval_loss
FROM
  ML.TRAINING_INFO(MODEL `corona.Classification_model_bq2`)
ORDER BY iteration ASC

In [ ]:
loss_curve2.plot(x='iteration');

In [ ]:
%%bigquery --project $project_id
SELECT roc_auc,
       CASE WHEN roc_auc > .85 THEN 'good'
            WHEN roc_auc > .7 THEN 'fair'
            WHEN roc_auc > .5 THEN 'not great'
            ELSE 'poor' END AS model_quality
FROM ML.EVALUATE(MODEL `corona.Classification_model_bq2`)

As you can see model performance has improved!

With BigQuery you can take advantage of an already available highly powerful computer data processing and analysis platform to build a machine learning model, without moving your data! You can learn more about BQML here. For our data set, you can build a DNN_Regression model to predict the result_score for a peptide HLA binding. Try that as a practice!

## Step 4: Run Prediction on BQML Model
Now that you have evaluated your model, the next step is to use it to predict
outcomes. 

To run the query that uses the model to predict the number of transactions:
Following example demonstrate leveraging BQ model for prediction. Optionally, you can export model and publish it on to Google AI Platform for serving prediction.


In [ ]:
%%bigquery --project $project_id
SELECT
  predicted_result_quality, predicted_result_quality_probs, result_quality as original_result, result_score
FROM ML.PREDICT(MODEL `corona.Classification_model_bq2`, (
  SELECT result_quality, Allele, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, result_score
  FROM `corona.mhc_qual_feature`
  WHERE length(peptide) IN (9,10)
  AND rand() < 0.0009))

The result shows predicted quality class with confidence. You can compare that with original result. Next step is to operationalize ML pipeline so that you can efficiently perform data updates and model updates. Check out AI Pipeline example for peptide prediction to learn more!